# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the batch_size here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0, 30)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    image_resized=imresize(image,(120,120,3)) #image height and width is taken as 120
                    
                    batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

#Defining the filters and dense layers
F = [8,16,32,64]
D = [256, 128, 5]

# Input shape
I=(30,120,120,3) # (frames,num_rows,num_cols,channel)

#Model Definition
model = Sequential()

model.add(Conv3D(F[0], 
                 kernel_size=(3,3,3), 
                 input_shape=I,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(F[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(F[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(F[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layer
model.add(Flatten())

model.add(Dense(D[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(D[1], activation='relu'))
model.add(Dropout(0.5))

#softmax
model.add(Dense(D[2], activation='softmax'))

Now that you have written the model, the next step is to compile the model. When you print the summary of the model, you'll see the total number of parameters you have to train.

In [7]:
from keras.optimizers import Adam
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the train_generator and the val_generator which will be used in .fit_generator.

In [8]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The steps_per_epoch and validation_steps are used by fit_generator to decide the number of next() calls it need to make.

In [10]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [11]:
x = 30 #Frame
y = 120 #rows
z = 120 #columns
batch_size = 10
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 81s 1s/step - loss: 1.8638 - categorical_accuracy: 0.2522 - val_loss: 1.4861 - val_categorical_accuracy: 0.3100

Epoch 00001: saving model to model_init_2020-05-2311_49_13.739767/model-00001-1.86382-0.25224-1.48615-0.31000.h5
Epoch 2/10
67/67 [==============================] - 25s 378ms/step - loss: 1.5868 - categorical_accuracy: 0.3134 - val_loss: 1.5479 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2020-05-2311_49_13.739767/model-00002-1.58680-0.31343-1.54794-0.21000.h5
Epoch 3/10
67/67 [==============================] - 26s 393ms/step - loss: 1.5820 - categorical_accuracy: 0.2687 - val_loss: 1.4048 - val_categorical_accuracy: 0.2900

Epoch 00003: saving model to model_init_2020-05-2311_49_13.739767/model-00003-1.58196-0.26866-1.40476-0.29000.h5
Epoch 4/10
67/67 [==============================] - 26s 394ms/step - loss: 1.5488 - categorical_accuracy: 0.3234 - val_loss: 1.5578 - val_categorical_accuracy: 0.1900

Epo

In [12]:
batch_size = 512
num_epochs = 90

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/90


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


2/2 [==============================] - 1s 734ms/step - loss: 1.1170 - categorical_accuracy: 0.5000 - val_loss: 1.0422 - val_categorical_accuracy: 0.5000

Epoch 00001: saving model to model_init_2020-05-2311_49_13.739767/model-00001-1.11703-0.50000-1.04217-0.50000.h5
Epoch 2/90
2/2 [==============================] - 2s 882ms/step - loss: 1.1353 - categorical_accuracy: 0.3333 - val_loss: 0.9956 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to model_init_2020-05-2311_49_13.739767/model-00002-1.13532-0.33333-0.99558-0.50000.h5
Epoch 3/90
2/2 [==============================] - 2s 878ms/step - loss: 1.3571 - categorical_accuracy: 0.5000 - val_loss: 0.8452 - val_categorical_accuracy: 0.9000

Epoch 00003: saving model to model_init_2020-05-2311_49_13.739767/model-00003-1.35713-0.50000-0.84521-0.90000.h5
Epoch 4/90
2/2 [==============================] - 2s 806ms/step - loss: 1.3138 - categorical_accuracy: 0.1667 - val_loss: 0.8652 - val_categorical_accuracy: 0.6000

Epoch 00004:


Epoch 00027: saving model to model_init_2020-05-2311_49_13.739767/model-00027-1.42989-0.33333-1.15905-0.50000.h5
Epoch 28/90
2/2 [==============================] - 2s 794ms/step - loss: 0.6214 - categorical_accuracy: 0.8333 - val_loss: 1.0872 - val_categorical_accuracy: 0.5000

Epoch 00028: saving model to model_init_2020-05-2311_49_13.739767/model-00028-0.62144-0.83333-1.08722-0.50000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.
Epoch 29/90
2/2 [==============================] - 1s 716ms/step - loss: 0.9968 - categorical_accuracy: 0.5000 - val_loss: 0.6667 - val_categorical_accuracy: 1.0000

Epoch 00029: saving model to model_init_2020-05-2311_49_13.739767/model-00029-0.99682-0.50000-0.66666-1.00000.h5
Epoch 30/90
2/2 [==============================] - 1s 671ms/step - loss: 1.3210 - categorical_accuracy: 0.3333 - val_loss: 1.0945 - val_categorical_accuracy: 0.6000

Epoch 00030: saving model to model_init_2020-05-2311_49_13.739767/model-00030-1.


Epoch 00053: saving model to model_init_2020-05-2311_49_13.739767/model-00053-0.74669-0.66667-0.77880-0.90000.h5
Epoch 54/90
2/2 [==============================] - 1s 723ms/step - loss: 1.2876 - categorical_accuracy: 0.3333 - val_loss: 0.9322 - val_categorical_accuracy: 0.6000

Epoch 00054: saving model to model_init_2020-05-2311_49_13.739767/model-00054-1.28756-0.33333-0.93220-0.60000.h5

Epoch 00054: ReduceLROnPlateau reducing learning rate to 5.960464760645934e-11.
Epoch 55/90
2/2 [==============================] - 1s 517ms/step - loss: 0.9439 - categorical_accuracy: 0.8333 - val_loss: 1.2231 - val_categorical_accuracy: 0.5000

Epoch 00055: saving model to model_init_2020-05-2311_49_13.739767/model-00055-0.94387-0.83333-1.22313-0.50000.h5
Epoch 56/90
2/2 [==============================] - 1s 645ms/step - loss: 1.4596 - categorical_accuracy: 0.3333 - val_loss: 0.9280 - val_categorical_accuracy: 0.8000

Epoch 00056: saving model to model_init_2020-05-2311_49_13.739767/model-00056-1.4

2/2 [==============================] - 2s 918ms/step - loss: 1.2445 - categorical_accuracy: 0.3333 - val_loss: 0.8909 - val_categorical_accuracy: 0.8000

Epoch 00079: saving model to model_init_2020-05-2311_49_13.739767/model-00079-1.24452-0.33333-0.89088-0.80000.h5
Epoch 80/90
2/2 [==============================] - 1s 723ms/step - loss: 1.0571 - categorical_accuracy: 0.8333 - val_loss: 1.1158 - val_categorical_accuracy: 0.5000

Epoch 00080: saving model to model_init_2020-05-2311_49_13.739767/model-00080-1.05708-0.83333-1.11583-0.50000.h5

Epoch 00080: ReduceLROnPlateau reducing learning rate to 7.275957959772868e-15.
Epoch 81/90
2/2 [==============================] - 1s 616ms/step - loss: 0.8250 - categorical_accuracy: 0.8333 - val_loss: 0.7633 - val_categorical_accuracy: 0.9000

Epoch 00081: saving model to model_init_2020-05-2311_49_13.739767/model-00081-0.82497-0.83333-0.76330-0.90000.h5
Epoch 82/90
2/2 [==============================] - 1s 693ms/step - loss: 0.5586 - categorical_

### Model - CNN - RNN (LSTM)

In [13]:
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.recurrent import LSTM

model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
          input_shape=(30,120,120,3)))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(MaxPooling2D((2,2))))

model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(MaxPooling2D((2,2))))

model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(MaxPooling2D((2,2))))

model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(MaxPooling2D((2,2))))

model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(MaxPooling2D((2,2))))

#Flatten Layer
model.add(TimeDistributed(Flatten()))

model.add(LSTM(64))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

#softmax
model.add(Dense(5, activation='softmax'))

In [14]:
optimiser = Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 30, 30, 64)    18496     
__________

Let us now fit the CNN-RNN model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [15]:
x = 30 #Frame
y = 120 #rows
z = 120 #columns
batch_size = 10
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


2/2 [==============================] - 5s 3s/step - loss: 1.9289 - categorical_accuracy: 0.3333 - val_loss: 1.6844 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2020-05-2311_49_13.739767/model-00001-1.92885-0.33333-1.68438-0.20000.h5
Epoch 2/10
2/2 [==============================] - 1s 317ms/step - loss: 2.4363 - categorical_accuracy: 0.0000e+00 - val_loss: 1.7785 - val_categorical_accuracy: 0.1000

Epoch 00002: saving model to model_init_2020-05-2311_49_13.739767/model-00002-2.43633-0.00000-1.77847-0.10000.h5
Epoch 3/10
2/2 [==============================] - 1s 311ms/step - loss: 1.3960 - categorical_accuracy: 0.5000 - val_loss: 1.7780 - val_categorical_accuracy: 0.1000

Epoch 00003: saving model to model_init_2020-05-2311_49_13.739767/model-00003-1.39605-0.50000-1.77803-0.10000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/10
2/2 [==============================] - 1s 313ms/step - loss: 1.9340 - categorical_ac

In [16]:
batch_size = 256
num_epochs = 60

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/60


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


3/3 [==============================] - 2s 708ms/step - loss: 1.3599 - categorical_accuracy: 0.3333 - val_loss: 1.6138 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2020-05-2311_49_13.739767/model-00001-1.35992-0.33333-1.61379-0.20000.h5
Epoch 2/60
3/3 [==============================] - 2s 596ms/step - loss: 2.3562 - categorical_accuracy: 0.0000e+00 - val_loss: 1.6054 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2020-05-2311_49_13.739767/model-00002-2.35621-0.00000-1.60541-0.20000.h5
Epoch 3/60
3/3 [==============================] - 2s 510ms/step - loss: 1.4416 - categorical_accuracy: 0.3333 - val_loss: 1.6695 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init_2020-05-2311_49_13.739767/model-00003-1.44158-0.33333-1.66949-0.20000.h5
Epoch 4/60
3/3 [==============================] - 2s 569ms/step - loss: 1.6750 - categorical_accuracy: 0.1111 - val_loss: 1.5856 - val_categorical_accuracy: 0.2000

Epoch 00

3/3 [==============================] - 2s 537ms/step - loss: 1.4060 - categorical_accuracy: 0.4444 - val_loss: 1.5266 - val_categorical_accuracy: 0.3000

Epoch 00028: saving model to model_init_2020-05-2311_49_13.739767/model-00028-1.40601-0.44444-1.52661-0.30000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 1.2207031829802872e-07.
Epoch 29/60
3/3 [==============================] - 1s 403ms/step - loss: 2.1586 - categorical_accuracy: 0.2222 - val_loss: 1.5575 - val_categorical_accuracy: 0.4000

Epoch 00029: saving model to model_init_2020-05-2311_49_13.739767/model-00029-2.15863-0.22222-1.55751-0.40000.h5
Epoch 30/60
3/3 [==============================] - 1s 497ms/step - loss: 1.7980 - categorical_accuracy: 0.1111 - val_loss: 1.6339 - val_categorical_accuracy: 0.1000

Epoch 00030: saving model to model_init_2020-05-2311_49_13.739767/model-00030-1.79795-0.11111-1.63387-0.10000.h5

Epoch 00030: ReduceLROnPlateau reducing learning rate to 6.103515914901436e-08.
Epoch 31/60


3/3 [==============================] - 2s 543ms/step - loss: 1.9156 - categorical_accuracy: 0.3333 - val_loss: 1.5101 - val_categorical_accuracy: 0.3000

Epoch 00054: saving model to model_init_2020-05-2311_49_13.739767/model-00054-1.91563-0.33333-1.51014-0.30000.h5
Epoch 55/60
3/3 [==============================] - 2s 550ms/step - loss: 1.7031 - categorical_accuracy: 0.2222 - val_loss: 1.6467 - val_categorical_accuracy: 0.1000

Epoch 00055: saving model to model_init_2020-05-2311_49_13.739767/model-00055-1.70310-0.22222-1.64674-0.10000.h5

Epoch 00055: ReduceLROnPlateau reducing learning rate to 2.980232380322967e-11.
Epoch 56/60
3/3 [==============================] - 2s 526ms/step - loss: 1.3128 - categorical_accuracy: 0.5556 - val_loss: 1.6306 - val_categorical_accuracy: 0.1000

Epoch 00056: saving model to model_init_2020-05-2311_49_13.739767/model-00056-1.31276-0.55556-1.63058-0.10000.h5
Epoch 57/60
3/3 [==============================] - 2s 585ms/step - loss: 1.7449 - categorical_